# Roberta - masked language modelling - Rupert AI

The Microsoft Research Sentence Completion Challenge - MRSCC (Zweig and Burges, 2011) requires a system to be able to predict which is the most likely word (from a set of 5 possibilities) to complete a sentence.

Roberta is trained on a masked language modelling task so it can complete this task straight out of the box! However, to get the best performance we should match our training distribution to our target distribution (sherlock holmes) - a great example of transfer learning.

* Training data: a training corpus of 19th century novels data (522 files) from Project Gutenberg data.
* Test data: 1040 sentences from sherlock holmes novels with one missing word and 5 options to choose from

In [ ]:
%%capture
!pip install transformers

# Data processing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import os
mrscc_dir = '/content/drive/MyDrive/mrscc/sentence-completion'
questions = pd.read_csv(os.path.join(mrscc_dir,"testing_data.csv"))
answers = pd.read_csv(os.path.join(mrscc_dir,"test_answer.csv"))

In [ ]:
answers

,id,answer
0,1,c
1,2,a
2,3,d
3,4,c
4,5,d
...,...,...
1035,1036,a
1036,1037,d
1037,1038,a
1038,1039,b


# Apply model to MLM task

N.B. Each candidate word is placed into each sentence - resulting in 5 candidate sentences. Each of these sentences are then tokenised and all candidate word **tokens** are masked. E.g. the flowers were placed *instantaenously* on the table will have several < mask > tokens as the token ID of *instantaenously* consists of several token IDs.

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM
import numpy as np
import re

choices = {'a)':1, 'b)':2, 'c)':3, 'd)':4, 'e)':5}
model_name = 'distilroberta-base'


In [ ]:
class LanguageModelEvaluator():

  def __init__(self, q, a, c, mn):
    self.questions, self.answers, self.choices, self.model_name = q, a, c, mn
    print(len(self.questions))
    self.process_questions_and_answers()
    self.tokenizer = AutoTokenizer.from_pretrained(model_name)
    self.model = AutoModelForMaskedLM.from_pretrained(model_name)
    self.sent_encodings, self.word_encodings, self.mask_idxs = self.make_encodings()

  def run_model_and_evaluate(self):
    output = self.make_predictions()
    self.accuracy = self.get_model_accuracy(output, self.questions['answer'])

  def process_questions_and_answers(self, s='_____'):
    answer_idxs, candidate_questions = [], []
    for index, row in self.questions.iterrows():
      answer = answers.iloc[index].answer + ')'
      answer_idxs.append(self.choices.get(answer))
      candidate_questions.append([re.sub(s, row.loc[c], row.loc['question']) for c in self.choices.keys()])
    self.questions.loc[:, 'candidate_questions'] = candidate_questions
    self.questions.loc[:, 'answer'] = answer_idxs

  def get_sublist_idxs_in_list(self, word, sentence):
    # find mask indicies for encoded sentence
    possibles = np.where(sentence == word[0])[0]
    for p in possibles:
      check = sentence[p:p + len(word)]
      if np.all(check == word):
          return list(range(p, (p + len(word))))

  def make_encodings(self):
    sent_encodings, word_encodings, mask_idxs = [], [], []
    for index, row in self.questions.iterrows():
        _sent_encodings, _word_encodings, _mask_idxs = [], [], []
        for i, (word, sentence) in enumerate(zip(row[self.choices.keys()], row.loc['candidate_questions'])):
          encoded_word = self.tokenizer.encode(str(" " + word), add_special_tokens=False)
          encoded_sent = self.tokenizer.encode_plus(sentence, add_special_tokens=True, return_tensors='pt',
                                                padding='max_length', max_length=128, return_attention_mask=True)
          tokens_to_mask_idx = self.get_sublist_idxs_in_list(np.array(encoded_word), np.array(encoded_sent['input_ids'][0]))
          encoded_sent['input_ids'][0][tokens_to_mask_idx] = self.tokenizer.mask_token_id
          _sent_encodings.append(encoded_sent)
          _word_encodings.append(encoded_word)
          _mask_idxs.append(tokens_to_mask_idx)
        sent_encodings.append(_sent_encodings)
        word_encodings.append(_word_encodings)
        mask_idxs.append(_mask_idxs)
    return sent_encodings, word_encodings, mask_idxs

  def make_predictions(self):
    output = []
    for q_idx, (w, s, m) in enumerate(zip(self.word_encodings, self.sent_encodings, self.mask_idxs)):
      print(f'Question {q_idx}')
      predictions = []
      candidate_input_ids = torch.stack([inp_ids['input_ids'].squeeze(0) for inp_ids in s])
      candidate_attention_masks = torch.stack([am['attention_mask'].squeeze(0) for am in s])
      candidate_logits = self.model(candidate_input_ids, attention_mask=candidate_attention_masks).logits
      for idx, (token, mask_idxs) in enumerate(zip(w, m)):
        mask_token_logits = candidate_logits[idx, mask_idxs, token]
        candidate_score = float(torch.mean(mask_token_logits))
        predictions.append(candidate_score)
      output.append(np.argmax(predictions) + 1)
    return output

  def get_model_accuracy(self, predictions, ground_truth):
    correct = 0
    for pred, gt in zip(predictions, ground_truth):
      if pred == gt:
        correct += 1
    return correct/len(ground_truth)



In [ ]:
questions = pd.read_csv(os.path.join(mrscc_dir,"testing_data.csv"))
answers = pd.read_csv(os.path.join(mrscc_dir,"test_answer.csv"))
evaluator = LanguageModelEvaluator(questions, answers, choices, model_name)

1040


In [ ]:
evaluator.run_model_and_evaluate()

Question 0
Question 1
Question 2
Question 3
Question 4
Question 5
Question 6
Question 7
Question 8
Question 9
Question 10
Question 11
Question 12
Question 13
Question 14
Question 15
Question 16
Question 17
Question 18
Question 19
Question 20
Question 21
Question 22
Question 23
Question 24
Question 25
Question 26
Question 27
Question 28
Question 29
Question 30
Question 31
Question 32
Question 33
Question 34
Question 35
Question 36
Question 37
Question 38
Question 39
Question 40
Question 41
Question 42
Question 43
Question 44
Question 45
Question 46
Question 47
Question 48
Question 49
Question 50
Question 51
Question 52
Question 53
Question 54
Question 55
Question 56
Question 57
Question 58
Question 59
Question 60
Question 61
Question 62
Question 63
Question 64
Question 65
Question 66
Question 67
Question 68
Question 69
Question 70
Question 71
Question 72
Question 73
Question 74
Question 75
Question 76
Question 77
Question 78
Question 79
Question 80
Question 81
Question 82
Question 83
Qu

In [ ]:
evaluator.accuracy

0.7038461538461539